In [ ]:
#%tensorflow_version 1.x
#!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 92 kB 155 kB/s 
     |████████████████████████████████| 42 kB 921 kB/s 
     |████████████████████████████████| 154 kB 24.7 MB/s 
     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 26.3 MB 2.2 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
import numpy as np
import matplotlib.pyplot as plt
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset
import random
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

#from cleverhans.compat import softmax_cross_entropy_with_logits
#from cleverhans.utils_keras import KerasModelWrapper





In [ ]:
# Load the CIFAR-10 Dataset
(x_train_victim, y_train_victim), (x_test_victim, y_test_victim), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train_victim shape: " + str(x_train_victim.shape) + "\n" + "x_train_victim size: " + str(x_train_victim.size) + "\n" + # this print statement is used for understanding what the CIFAR-10 dataset is
      "y_train_victim shape: " + str(y_train_victim.shape) + "\n" + "y_train_victim size: " + str(y_train_victim.size) + "\n" +
      "x_test_victim shape: " + str(x_test_victim.shape) + "\n" + "x_test_victim size: " + str(x_test_victim.size) + "\n" +
      "y_test_victim shape: " + str(y_test_victim.shape) + "\n" + "y_test_victim size: " + str(y_test_victim.size) + "\n")
print()


#  Load the victim model
classifier_url ="https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1" #@param {type:"string"} # model is downloaded from this site
IMAGE_SHAPE = (32, 32) # the image shape is needed so that the model knows the input-shape and since we are working with the CIFAR-10 all the images are 32 x 32 color images
victim_classifier = KerasClassifier(model=tf.keras.Sequential([hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))]), clip_values=(min_, max_)) # this bascially creates a keras wrapper around the downloaded model so that we can use it with keras functions.


# Evaluating the victim model on the benign dataset
predictions = victim_classifier.predict(x_test_victim) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_victim, axis=1)) / len(y_test_victim) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


In [ ]:
#  Collect subset
def exract_subset(data, labels):
    x_pre = []  
    y_pre = []  
    #count = 0
    for index in range(0, len(data)):  
        x_pre.append(data[index])  
        y_predict = np.argmax(victim_classifier.predict( data[index].reshape( (1, data[index].shape[ 0 ], data[index].shape[ 1 ], data[index].shape[ 2 ]) ) ) )  # add the image label to the y_test set
        y_pre.append(y_predict)
       
    x = np.asarray(x_pre)  
    y = keras.utils.to_categorical(np.asarray(y_pre), 10)  
    return x, y

X_subset, Y_subset = exract_subset( x_test_victim, y_test_victim )
print("X_subset shape: " + str(X_subset.shape) + "\n" + "X_subset size: " + str(X_subset.size) + "\n" +
      "Y_subset shape: " + str(Y_subset.shape) + "\n" + "Y_subset size: " + str(Y_subset.size) + "\n")

X_subset shape: (10000, 32, 32, 3)
X_subset size: 30720000
Y_subset shape: (10000, 10)
Y_subset size: 100000



In [ ]:
!apt-get install cleverhans

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package cleverhans


Up to this point the victim classifer has been evaluted on the test set of cifar10 = 94.5% accuracy, and the test set has been extracted with the labels given by the victim model's predtiction, i.e. 10000 examples

In [ ]:
X =X_subset
Y = Y_subset
x_train_substitute = X[:7500]
y_train_substitute = Y[:7500]
print("x_train_substitute shape: " + str(x_train_substitute.shape) + "\n" + "x_train_substitute size: " + str(x_train_substitute.size) + "\n" + 
      "y_train_substitute shape: " + str(y_train_substitute.shape) + "\n" + "y_train_substitute size: " + str(y_train_substitute.size) + "\n")
print()

x_test_substitute = X[7500:]
y_test_substitute = Y[7500:]
print("x_test_substitute shape: " + str(x_test_substitute.shape) + "\n" + "x_test_substitute size: " + str(x_test_substitute.size) + "\n" + 
      "y_test_substitute shape: " + str(y_test_substitute.shape) + "\n" + "y_test_substitute size: " + str(y_test_substitute.size) + "\n")
print()


#  Creating the model
model_substitute = Sequential()
model_substitute.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train_substitute.shape[1:]))
model_substitute.add(Activation("relu"))
model_substitute.add(Conv2D(32, (3, 3)))
model_substitute.add(Activation("relu"))
model_substitute.add(MaxPooling2D(pool_size=(2, 2)))
model_substitute.add(Dropout(0.25))

model_substitute.add(Conv2D(64, (3, 3), padding="same"))
model_substitute.add(Activation("relu"))
model_substitute.add(Conv2D(64, (3, 3)))
model_substitute.add(Activation("relu"))
model_substitute.add(MaxPooling2D(pool_size=(2, 2)))
model_substitute.add(Dropout(0.25))

model_substitute.add(Flatten())
model_substitute.add(Dense(512))
model_substitute.add(Activation("relu"))
model_substitute.add(Dropout(0.5))
model_substitute.add(Dense(10))
model_substitute.add(Activation("softmax"))

model_substitute.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



# Step 3: Creating the classifier
substitute_classifier = KerasClassifier(model=model_substitute, clip_values=(0., 1.))


x_train_substitute shape: (7500, 32, 32, 3)
x_train_substitute size: 23040000
y_train_substitute shape: (7500, 10)
y_train_substitute size: 75000


x_test_substitute shape: (2500, 32, 32, 3)
x_test_substitute size: 7680000
y_test_substitute shape: (2500, 10)
y_test_substitute size: 25000





In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def one_hot(value):
  vec = np.zeros((10))
  vec[value] = 1
  return vec

def jacobian(predictions, inputs, num_classes):
    #That is, how does the kth element of yhat vary wrt x?
    return [tf.gradients(predictions[:, c], inputs)[0] for c in range(0, num_classes)]

def jacobian_prediction_dimension(grads, predictions):
    return [grads[predictions[i]][i] for i in np.arange(len(predictions))]

wrapper = KerasClassifier( model_substitute ) # create keras wrapper for substitute model
adv_train_epochs = 1 # run the loop for 2 runs genreating 22500 samples from 7500 samples
adv_train_set = x_train_substitute / 255.0 # assign the varaible adv_train_set the x_train which holds 7500 samples


for adv_train_epoch in range(adv_train_epochs): # loop for syntehtic data generation
    print("RUN: " + str(adv_train_epoch))
    print("Before: ")
    print(adv_train_set.shape) # initial size of the x_train (7500, 32, 32, 3)

    # Get labels from victim model and train substitute model
    oracle_labels = victim_classifier.predict(adv_train_set) # have the victim model label the x_train values, i.e. getting the y_train labels = 7500 labels
    print("Oracle Predict: ")
    print(oracle_labels.shape) # shape of the y_train = (7500, 10)
    substitute_classifier.fit(adv_train_set, oracle_labels, nb_epochs=10, batch_size=128) # fit the substitute model with the training set created thus far.    

    # Convert the labels from victim model to one hot encoded vectors
    oracle_labels = np.zeros((adv_train_set.shape[0],10))
    for i in range(0,x_train_substitute.shape[0]):
        oracle_labels[i] = one_hot(np.argmax(victim_classifier.predict( adv_train_set[i].reshape( (1, adv_train_set[i].shape[ 0 ], adv_train_set[i].shape[ 1 ], adv_train_set[i].shape[ 2 ]) ) )))
    print("Inside Augment: " + str(oracle_labels.shape))
    


RUN: 0
Before: 
(7500, 32, 32, 3)
Oracle Predict: 
(7500, 10)
Epoch 1/10
7500/7500 [==============================] - 19s 3ms/step - loss: -4681832998268008.0000 - accuracy: 0.9863
Epoch 2/10
7500/7500 [==============================] - 19s 3ms/step - loss: -8404932867181074.0000 - accuracy: 0.9875
Epoch 3/10
7500/7500 [==============================] - 19s 3ms/step - loss: -14249379158178206.0000 - accuracy: 0.9876
Epoch 4/10
7500/7500 [==============================] - 19s 3ms/step - loss: -23043457454749988.0000 - accuracy: 0.9864
Epoch 5/10
7500/7500 [==============================] - 19s 3ms/step - loss: -35760463533504848.0000 - accuracy: 0.9877
Epoch 6/10
7500/7500 [==============================] - 19s 3ms/step - loss: -53634649193476648.0000 - accuracy: 0.9881
Epoch 7/10
7500/7500 [==============================] - 19s 3ms/step - loss: -77940428083473776.0000 - accuracy: 0.9881
Epoch 8/10
7500/7500 [==============================] - 19s 3ms/step - loss: -110631488075768848.000

In [ ]:
    # Create a session
    with tf.Session() as sess:
        xm = model_substitute.layers[0].input 
        yhat = wrapper.get_logits( xm )
        init = tf.global_variables_initializer()
        sess = tf.Session( )   
        sess.run(init) # Initializes the variables
        grads = sess.run(jacobian(yhat, xm, 10), feed_dict={xm: adv_train_set}) # compute the grads with the jacobian function
        jpd = jacobian_prediction_dimension(grads, np.argmax(oracle_labels, 1))

    perturbed_set = []
    jbda_lambda = 0.1
    tau = 1
    jbda_epoch_lambda = jbda_lambda * np.power(-1, np.floor(adv_train_epoch/tau))
    for idx, example in enumerate(adv_train_set):
        new_example = example + jbda_epoch_lambda * (np.sign(jpd[idx]))
        perturbed_set.append(new_example)
    adv_train_set = np.vstack((adv_train_set, np.array(perturbed_set)))

    print("After: ")
    print(adv_train_set.shape)

In [ ]:
print(adv_train_set.shape)

(15000, 32, 32, 3)


In [ ]:
# Collect subset
def exract_subset(data):
    x_pre = []  
    y_pre = []  
    #count = 0
    for index in range(0, len(data)):  
        x_pre.append(data[index])  
        y_predict = np.argmax(victim_classifier.predict( data[index].reshape( (1, data[index].shape[ 0 ], data[index].shape[ 1 ], data[index].shape[ 2 ]) ) ) )  # add the image label to the y_test set
        y_pre.append(y_predict)
        #if y_predict != np.argmax(labels[index]):
            #print(str(np.argmax(labels[index])) + " : " + str(y_predict))
            #count = count + 1
    #print(count)
    x = np.asarray(x_pre)  
    y = keras.utils.to_categorical(np.asarray(y_pre), 10)  
    return x, y

X_synthetic, Y_synthetic = exract_subset( adv_train_set )
print("X_synthetic shape: " + str(X_synthetic.shape) + "\n" + "X_synthetic size: " + str(X_synthetic.size) + "\n" +
      "Y_synthetic shape: " + str(Y_synthetic.shape) + "\n" + "Y_synthetic size: " + str(Y_synthetic.size) + "\n")

#np.save('/content/gdrive/My Drive/X_synthetic', X_synthetic) 
#np.save('/content/gdrive/My Drive/Y_synthetic', Y_synthetic) 

X_synthetic shape: (7500, 32, 32, 3)
X_synthetic size: 23040000
Y_synthetic shape: (7500, 10)
Y_synthetic size: 75000



In [ ]:
#X_train = np.load('/content/gdrive/My Drive/X_synthetic.npy') 
#Y_train = np.load('/content/gdrive/My Drive/Y_synthetic.npy') 
#print("X shape: " + str(X.shape) + "\n" + "X size: " + str(X.size) + "\n" + 
#      "Y shape: " + str(Y.shape) + "\n" + "Y size: " + str(Y.size) + "\n")
#print()
X_train = X_synthetic
Y_train = Y_synthetic

substitute_classifier.fit(X_train, Y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
7500/7500 [==============================] - 19s 2ms/step - loss: 31304155450.5728 - accuracy: 0.9897
Epoch 2/10
7500/7500 [==============================] - 19s 2ms/step - loss: 34639197573.3931 - accuracy: 0.9888
Epoch 3/10
7500/7500 [==============================] - 19s 2ms/step - loss: 29232477993.9157 - accuracy: 0.9915
Epoch 4/10
7500/7500 [==============================] - 19s 2ms/step - loss: 21465745466.0267 - accuracy: 0.9925
Epoch 5/10
7500/7500 [==============================] - 19s 2ms/step - loss: 22619373681.5957 - accuracy: 0.9936
Epoch 6/10
7500/7500 [==============================] - 19s 2ms/step - loss: 12121543033.9243 - accuracy: 0.9951
Epoch 7/10
7500/7500 [==============================] - 19s 2ms/step - loss: 11402851839.7269 - accuracy: 0.9952
Epoch 8/10
7500/7500 [==============================] - 19s 2ms/step - loss: 8420498132.1728 - accuracy: 0.9965
Epoch 9/10
7500/7500 [==============================] - 18s 2ms/step - loss: 7832588692.1387 - ac

In [ ]:
#Evaluate the ART classifier on benign test examples
predictions = substitute_classifier.predict(x_test_substitute)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_substitute, axis=1)) / len(y_test_substitute)
print("Accuracy on benign test examples for substitute classifier: {}%".format(accuracy * 100))

Accuracy on benign test examples for substitute classifier: 10.639999999999999%


In [ ]:
substitute_classifier.save("/content/gdrive/My Drive/Attack_Transfer_Substitute_Classifier_V3.h5")

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import art
from art.classifiers import KerasClassifier
tf.compat.v1.disable_eager_execution()

In [ ]:

from art.attacks import FastGradientMethod

fgsm = FastGradientMethod(classifier=substitute_classifier)
x_adversarial_fgsm = fgsm.generate(x_train_substitute,y_train_substitute)


In [ ]:
predictions = substitute_classifier.predict(x_test_substitute)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_substitute, axis=1)) / len(y_test_substitute)
print("Accuracy on original examples for substitute classifier: {}%".format(accuracy))

Accuracy on original examples for substitute classifier: 0.1064%


In [ ]:
predictions = substitute_classifier.predict(x_adversarial_fgsm)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute)
print("Accuracy on adversarial exales for substitute classifier:{}".format(accuracy))

Accuracy on adversarial exales for substitute classifier:0.09906666666666666


In [ ]:
victim_classifier = KerasClassifier(model=model_substitute, clip_values=(0., 1.))

In [ ]:
predictions = victim_classifier.predict(x_adversarial_fgsm) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


Accuracy on benign test examples for victim classifier: 9.906666666666666%



In [ ]:
from art.attacks import SaliencyMapMethod

sm = SaliencyMapMethod(classifier=substitute_classifier)
x_adversarial_sm = fgsm.generate(x_train_substitute,y_train_substitute)

In [ ]:
predictions = substitute_classifier.predict(x_adversarial_sm)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute)
print("Accuracy on original examples for substitute classifier: {}%".format(accuracy*100))

Accuracy on original examples for substitute classifier: 9.906666666666666%
